In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/ayushjha/Desktop/sih ml_model/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <636BF463-1886-392D-B8B3-6011C44DCEE9> /Users/ayushjha/Desktop/sih ml_model/.venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/o

In [ ]:
df = pd.read_excel("final_model_v1.xlsx")
df.head()

,student_id,scenario,sem_no,subject_code,credit,t1,t2,t3,endsem,attendance_pct,fee_amount_due,fee_days_late,attempts,dropout
0,S001,early_transfer,1,SUB01,4,14,15,-1,31,61,0,0,1,1
1,S001,early_transfer,1,SUB02,4,14,16,10,26,87,0,0,1,1
2,S001,early_transfer,1,SUB03,4,14,13,15,38,81,0,0,1,1
3,S001,early_transfer,1,SUB04,2,14,14,14,41,65,0,0,1,1
4,S001,early_transfer,1,SUB05,5,14,8,7,47,79,0,0,1,1


In [5]:
df.head()

,student_id,scenario,sem_no,subject_code,credit,t1,t2,t3,endsem,attendance_pct,fee_amount_due,fee_days_late,attempts,dropout
0,S001,early_transfer,1,SUB01,4,14,15,-1,31,61,0,0,1,1
1,S001,early_transfer,1,SUB02,4,14,16,10,26,87,0,0,1,1
2,S001,early_transfer,1,SUB03,4,14,13,15,38,81,0,0,1,1
3,S001,early_transfer,1,SUB04,2,14,14,14,41,65,0,0,1,1
4,S001,early_transfer,1,SUB05,5,14,8,7,47,79,0,0,1,1


In [10]:
df = df.rename(columns = {"t1" : "M1" , "t2" : "M2" , "t3": "M3" , "endsem" : "EndSem" , "attendance_pct": "Attendance Percentage"})
df.head()

,student_id,scenario,sem_no,subject_code,credit,M1,M2,M3,EndSem,Attendance Percentage,fee_amount_due,fee_days_late,attempts,dropout
0,S001,early_transfer,1,SUB01,4,14,15.0,NaN,31.0,61,0,0,1,1
1,S001,early_transfer,1,SUB02,4,14,16.0,10.0,26.0,87,0,0,1,1
2,S001,early_transfer,1,SUB03,4,14,13.0,15.0,NaN,81,0,0,1,1
3,S001,early_transfer,1,SUB04,2,14,14.0,14.0,41.0,65,0,0,1,1
4,S001,early_transfer,1,SUB05,5,14,NaN,7.0,47.0,79,0,0,1,1


In [11]:
def simulate_missing(df):
    for col in ["M2", "M3", "EndSem"]:
        mask = np.random.rand(len(df)) < 0.30   # randomly remove 30%
        df.loc[mask, col] = np.nan
    return df

In [12]:
df = simulate_missing(df)

In [ ]:
X = df[["M1", "M2", "M3", "EndSem", "Attendance Percentage", "credit" ,"fee_days_late" , "attempts" , "sem_no"]]

y = df["dropout"]

In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [78]:
model = XGBClassifier(
    scale_pos_weight = 2,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    missing=np.nan
)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [68]:
preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

Accuracy: 0.9887152777777778


In [ ]:
def predict_risk(t1=None, t2=None, t3=None, endsem = None, attendance=None , credit = None ,fee_days_late= None , attempts = None , sem_no = None):

    # Convert missing to NaN
    row = {
        "M1": t1,
        "M2": t2 if t2 is not None else np.nan,
        "M3": t3 if t3 is not None else np.nan,
        "EndSem" : endsem if endsem is not None else np.nan,
        "Attendance Percentage": attendance,
        "credit" : credit,
        "fee_days_late": fee_days_late,
        "attempts" : attempts,
        "sem_no" : sem_no
    }

    df_row = pd.DataFrame([row])

    prob = float(model.predict_proba(df_row)[0][1])   # probability student dropout
    risk = int(model.predict(df_row)[0])              # 1 = high risk, 0 = safe

    return risk, prob

In [81]:
# drop, risk_factor = predict_dropout1(20,100)
# print("Drop Out : ", drop)
# print("Risk Factor : ", 1 -risk_factor)

In [86]:
print((predict_risk(t1 = 2 , t2 = -2 ,t3 = 15 , endsem = 6 ,attendance = 45 ,credit = 5 , fee_days_late = 0 , attempts = 2 , sem_no = 8))[1])

0.9608819484710693


In [85]:
print(predict_risk(t1 = 0 , t2 = -2 ,t3 = 5 , endsem = 20 ,attendance = 40 , credit = 4 , fee_days_late= 33 , attempts = 1, sem_no =2)[1])

0.9733138680458069
